In [1]:
import numpy as np
import panel as pn
import pandas as pd
import holoviews as hv
pn.extension('echarts')
from time import sleep
import shutup
shutup.please()
from random import choice

In [5]:
# dataset for the timeline graph
dftest = pd.read_csv('china_cities_GDP.csv')
dftest.head()

,city,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,Beijing,4315.00,5007.21,6033.21,6969.52,8117.78,9846.81,11115.00,12153.03,14113.58,16251.93
1,Tianjin,2150.76,2578.03,3110.97,3905.64,4462.74,5252.76,6719.01,7521.85,9224.46,11307.28
2,Hebei,6018.28,6921.29,8477.63,10012.11,11467.60,13607.32,16011.97,17235.48,20394.26,24515.76
3,Shanxi,2324.80,2855.23,3571.37,4230.53,4878.61,6024.45,7315.40,7358.31,9200.86,11237.55
4,Inner Mongolia,1940.94,2388.38,3041.07,3905.03,4944.25,6423.18,8496.20,9740.25,11672.00,14359.88


In [130]:
# timeline graph
def timeline(df,title='',subtitle='',xtitle=None,ytitle=''):
    yearslist = df.columns.tolist()[1:]
    xtitle = xtitle if xtitle else df.columns.tolist()[0]
    yearslist = df.columns.tolist()[1:]
    citylist = df.iloc[:,0].to_list()
    dfvalues = df.iloc[:,1:].values.tolist()
    options = []
    for year in yearslist:
        tempo = {"title" :{"text": f'{year} {title}'},
                "series": [
                {"data": df[year].to_list()}
                ]
                }
        options.append(tempo)
    echart = {
        'baseOption': {
            'timeline': {
                'axisType': 'category',
                'autoPlay': True,
                'playInterval': 1000,
                'data': yearslist
            },
            'title': {
                'subtext': f'{subtitle}',
            },
            'tooltip': {},
            'legend':{
                'left': 'right',
                },
            'calculable' : True,
            'grid': {
                'top': 80,
                'bottom': 100
            },
            'xAxis': [
                {
                    'type':'category',
                    'axisLabel':{'interval':0, 'rotate': 45},
                    'data': citylist,
                    'splitLine': {'show': False},
                    'name': f'{xtitle}',
                }
            ],
            'yAxis': [
                {
                    'type': 'value',
                    'name': f'{ytitle}',
                    'max': 'dataMax',
                }
            ],
            'series': [
                {'name': 'GDP',
                 'type': 'bar',
                 # 'realtimeSort': True, # comment for normal chat
                },
                ]
        },
            'options': options,

        };
    
    return echart

timeline_dict = timeline(dftest,title='GDP',subtitle='hell yeah baby',xtitle='City',ytitle='$')

echart_pane = pn.pane.ECharts(timeline_dict, height=600, width=1000, theme='default')
echart_pane

ECharts(dict, height=600, sizing_mode='fixed', width=1000)

In [98]:
# dataset for the line race and bar race graphs

df = pd.read_json('life-expectancy-table.json')
df.columns = ['Income','Life Expectancy','Population','Country','Year']
df.dropna(inplace=True)
df_flags = pd.read_json('flags.json')
df_flags.columns = ["code","emoji","unicode","name","title","dialCode"]
df = df.merge(df_flags,how='inner',left_on='Country',right_on='name')
df = df[['Income','Life Expectancy','Population','Country','Year','emoji']]
df.sort_values(['Country', 'Year'],inplace=True)
hex_codes = '0123456789abcdef'
colors = []
items = small_df['Country'].unique().tolist()
for i in range(len(items)):
    combination = '#' + ''.join((choice(hex_codes), choice(hex_codes), choice(hex_codes), choice(hex_codes), choice(hex_codes), choice(hex_codes)))
    colors.append(combination)

color_dict = {i:j for i,j in zip(items,colors)}
df['color'] = df['Country'].map(color_dict)
small_df = df[df['Country'].isin(['Finland','France','Germany','Iceland','Norway','Poland','Russia','Australia','Japan','Turkey','Canada'])]
small_df.head()

,Income,Life Expectancy,Population,Country,Year,emoji,color
1372,17908,76,73517002,Turkey,2011,🇹🇷,#503b6c
1373,18057,76.2,74849187,Turkey,2012,🇹🇷,#503b6c
1374,18579,76.3,76223639,Turkey,2013,🇹🇷,#503b6c
1375,18884,76.4,77523788,Turkey,2014,🇹🇷,#503b6c
1376,19360,76.5,78665830,Turkey,2015,🇹🇷,#503b6c


In [123]:
#line race graph
x= 'Year'
y= 'Income'
filter_col= 'Country'
datasetWithFilters = []
seriesList = []
items = small_df[filter_col].unique().tolist()
chunks = sorted(small_df[x].unique().tolist())
for item in items:
    filtereddf = small_df[(df[filter_col] == item) & (small_df[x] == chunks[0])]
    datasetId = 'dataset_' + item
    datasetWithFilters.append({
      'id': datasetId,
      'source': [list(filtereddf.columns)] + filtereddf.values.tolist(),
    })
    seriesList.append({
      'type': 'line',
      'datasetId': datasetId,
      'showSymbol': False,
      'name': item,
      'endLabel': {
        'show': True,
        'formatter': '{a}: {@%s}' % y
      },
      'labelLayout': {
        'moveOverlap': 'shiftY'
      },
      'emphasis': {
        'focus': 'series'
      },
      'encode': {
        'x': x,
        'y': y,
        'label': [filter_col, y],
        'tooltip': [y],
      }
    });
option = {
'animationDuration': 10000,
'dataset': [*datasetWithFilters],
'title': {
  'text': 'Income of Germany and France since 1950'
},
'tooltip': {
  'order': 'valueDesc',
  'trigger': 'axis'
},
'xAxis': {
  'type': 'category',
  'nameLocation': 'middle',
},
'yAxis': {
  'name': y,
},
'grid': {
  'right': 140
},
'series': seriesList
};


def update_graph():
    for i, chunck in enumerate(chunks[:-1]):
        datasetWithFilters=[]
        for item in items:
            filtereddf = small_df[(small_df[filter_col] == item) & (small_df[x].isin(chunks[:i+1]))]
            datasetId = 'dataset_' + item
            datasetWithFilters.append({
              'id': datasetId,
              'source': [list(filtereddf.columns)] + filtereddf.values.tolist(),
            })
        echart_pane.object['dataset'] = [*datasetWithFilters]
        echart_pane.param.trigger('object')
        sleep(0.25)

button = pn.widgets.Button(name='► Play', width=60, align='end')

def animate(event):
    if button.name == '► Play':
        button.button_type = 'primary'
        button.name = 'Racing'
        update_graph()
        button.button_type = 'success'
        button.name = 'Completed'
    else:
        button.button_type = 'default'
        button.name = '► Play'

button.on_click(animate)


echart_pane = pn.pane.ECharts(option, height=400, width=1000, theme='light')
layout = pn.Column(echart_pane, button,sizing_mode='stretch_both')
layout

Column(sizing_mode='stretch_both')
    [0] ECharts(dict, height=400, sizing_mode='fixed', theme='light', width=1000)
    [1] Button(align='end', name='► Play', width=60)

In [126]:
# bar race graph
x= 'Year'
y= 'Income'
filter_col= 'Country'
datasetWithFilters = []
seriesList = []
chunks = sorted(small_df[x].unique().tolist())
items = small_df[filter_col].unique().tolist()
filtereddf = small_df[(small_df[x] == chunks[0])]
datasetId = 'dataset_' + str(chunks[0])
datasetWithFilters.append({
  'id': datasetId,
  'source': [list(filtereddf.columns)] + filtereddf.values.tolist(),
})
seriesList.append({
  'realtimeSort': True,
  'type': 'bar',
  'datasetId': datasetId,
  'showSymbol': False,
  'name': chunks[0],
  'labelLayout': {
    'moveOverlap': 'shiftY'
  },
  'emphasis': {
    'focus': 'series'
  },
  'encode': {
    'x': y,
    'y': filter_col,
    'label': {
          'show': True,
          'position': 'right',
          'valueAnimation': True,
        },  },
});
option = {
'animationDuration': 10000,
'dataset': [*datasetWithFilters],
'title': {
'text': 'Income since 1950'
},
'tooltip': {
'order': 'valueDesc',
'trigger': 'axis'
},
'xAxis': {
'type': 'value',
'nameLocation': 'middle',
'max': 'dataMax',
},
'yAxis': {
'type': 'category',
'inverse': True,
'max': 10,
'axisLabel': {
        'show': True,
        'fontSize': 14,
        'formatter': '{value}',
      },
'animationDuration': 300,
'animationDurationUpdate': 300
},
'grid': {
      'top': 30,
      'bottom': 30,
      'left': 150,
      'right': 80
    },
'animationDuration': 0,
'animationDurationUpdate': 2000,
'animationEasing': 'linear',
'animationEasingUpdate': 'linear',
'graphic': {
  'elements': [
    {
      'type': 'text',
      'right': 160,
      'bottom': 60,
      'style': {
        'text': chunks[0],
        'font': 'bolder 80px monospace',
        'fill': 'rgba(100, 100, 100, 0.25)'
      },
      'z': 100
    }
  ]
},
'series': seriesList
};
echart_pane = pn.pane.ECharts(option, height=400, width=1000, theme='light')
button = pn.widgets.Button(name='► Play', width=60, align='end')

def animate(event):
    if button.name == '► Play':
        button.button_type = 'primary'
        button.name = 'Racing'
        update_graph()
        button.button_type = 'success'
        button.name = 'Completed'
    else:
        button.button_type = 'default'
        button.name = '► Play'

button.on_click(animate)


def update_graph():
    for chunk in chunks:
        datasetWithFilters=[]
        seriesList = []
        filtereddf = small_df[(small_df[x] == chunk)]
        datasetId = 'dataset_' + str(chunk)
        datasetWithFilters.append({
          'id': datasetId,
          'source': [list(filtereddf.columns)] + filtereddf.values.tolist(),
        })
        seriesList.append({
        'realtimeSort': True,
        'type': 'bar',
        'color': colors,
        # 'itemStyle': {
        #   'color': b,
        # },
        'datasetId': datasetId,
        'showSymbol': False,
        'name': chunk,
        'labelLayout': {
        'moveOverlap': 'shiftY'
        },
        'emphasis': {
        'focus': 'series'
        },
        'encode': {
        'x': y,
        'y': filter_col,
        'label': {
          'show': True,
          'precision': 1,
          'position': 'right',
          'valueAnimation': True,
          'fontFamily': 'monospace'
        },
        },
        })
        option = {
        'animationDuration': 10000,
        'dataset': [*datasetWithFilters],
        'title': {
        'text': 'Income since 1950'
        },
        'tooltip': {
        'order': 'valueDesc',
        'trigger': 'axis'
        },
        'xAxis': {
        'type': 'value',
        'nameLocation': 'middle',
        'max': 'dataMax',
        },
        'yAxis': {
        'type': 'category',
        'inverse': True,
        'max': 10,
        'axisLabel': {
        'show': True,
        'fontSize': 14,
        'formatter': '{value}',
        },
        'animationDuration': 300,
        'animationDurationUpdate': 300,
        },
        'grid': {
      'top': 30,
      'bottom': 30,
      'left': 150,
      'right': 80
        },
        'animationDuration': 0,
        'animationDurationUpdate': 2000,
        'animationEasing': 'linear',
        'animationEasingUpdate': 'linear',
        'graphic': {
        'elements': [
        {
          'type': 'text',
          'right': 160,
          'bottom': 60,
          'style': {
            'text': chunk,
            'font': 'bolder 80px monospace',
            'fill': 'rgba(100, 100, 100, 0.25)'
          },
          'z': 100
        }
        ]
        },
        'series': seriesList
        };
        echart_pane.object = option
        echart_pane.param.trigger('object')
        sleep(0.25)        

layout = pn.Column(echart_pane, button,sizing_mode='stretch_both')
layout

Column(sizing_mode='stretch_both')
    [0] ECharts(dict, height=400, sizing_mode='fixed', theme='light', width=1000)
    [1] Button(align='end', name='► Play', width=60)